<a href="https://colab.research.google.com/github/lillanieder/data-projects/blob/main/4_lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
#data = pd.read_csv('https://raw.githubusercontent.com/lillanieder/data-projects/main/data/hatos.csv', header = None, sep = ';', names = list(range(20)))
data = pd.read_csv('https://raw.githubusercontent.com/lillanieder/data-projects/main/data/otos.csv', header = None, sep = ';')
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2023,6,2023.02.11.,0,0 Ft,22,2 285 905 Ft,1904,27 560 Ft,55687,3 180 Ft,10,32,40,70,82
1,2023,5,2023.02.04.,0,0 Ft,42,1 142 265 Ft,2855,17 535 Ft,77006,2 195 Ft,8,15,20,32,34
2,2023,4,2023.01.28.,0,0 Ft,19,2 411 060 Ft,1815,26 385 Ft,53591,3 110 Ft,20,51,54,59,86
3,2023,3,2023.01.21.,0,0 Ft,20,2 337 910 Ft,2315,21 115 Ft,62955,2 635 Ft,11,36,39,68,84
4,2023,2,2023.01.14.,0,0 Ft,33,1 572 215 Ft,2381,22 830 Ft,70141,2 710 Ft,16,25,32,72,85


In [3]:
#l = data.loc[:,13:18].values.tolist()
l = data.loc[:,11:15].values.tolist()
flat_list = [item for sublist in l for item in sublist]

In [4]:
from collections import Counter
counter = Counter(flat_list)
counter.most_common(15)

[(3, 229),
 (15, 219),
 (75, 219),
 (1, 217),
 (42, 214),
 (29, 214),
 (86, 211),
 (66, 210),
 (12, 209),
 (77, 209),
 (13, 209),
 (10, 206),
 (55, 206),
 (56, 206),
 (4, 205)]

In [ ]:
pd.DataFrame(sorted(counter.items())).set_index(0)[1].plot()

# Modeling

https://medium.com/@polanitzer/how-to-guess-accurately-3-lottery-numbers-out-of-6-using-lstm-model-e148d1c632d6

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [ ]:
df = data.loc[:,11:15]
df.head()

In [ ]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()

In [ ]:
# All our games
number_of_rows = df.values.shape[0]
print(number_of_rows)

# Amount of games we need to take into consideration for prediction
window_length = 7

# Balls counts
number_of_features = df.values.shape[1]
print(number_of_features)

In [ ]:
X = np.empty([number_of_rows - window_length, window_length, number_of_features],
             dtype=float)
y = np.empty([number_of_rows - window_length, number_of_features],
             dtype=float)

for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# first scaled sample (which made of 7 consecutive lottery games)
X[0]

In [ ]:
# first scaled label
y[0]

In [ ]:
# initialise RNN
model = Sequential()

# add the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                             input_shape = (window_length, number_of_features),
                             return_sequences = True)))

# add a first Dropout layer in order to reduce overfitting
model.add(Dropout(0.2))

# add a second LSTM layer
model.add(Bidirectional(LSTM(240,
                             input_shape = (window_length, number_of_features),
                             return_sequences = True)))

# add a second Dropout layer
model.add(Dropout(0.2))

# add a third LSTM layer
model.add(Bidirectional(LSTM(240,
                             input_shape = (window_length, number_of_features),
                             return_sequences = True)))

# add a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                             input_shape = (window_length, number_of_features),
                             return_sequences = False)))

# add a dense layer
model.add(Dense(59))

# add the last output layer
model.add(Dense(number_of_features))
# compile the RNN
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [ ]:
# train our LSTM model
model.fit(x=X, y=y, batch_size=100, epochs=300, verbose=2)

In [ ]:
to_predict = df.tail(8)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict = np.array(to_predict)

scaled_to_predict = scaler.transform(to_predict)

prediction = df.tail(1)

In [ ]:
y_pred = model.predict(np.array([scaled_to_predict]))
print('The predicted numbers in the last lottery game are:', scaler.inverse_transform(y_pred).astype(int)[0])

In [ ]:
prediction = np.array(prediction)
print('The actual numbers in the last lottery game were:', prediction[0])